In [1]:
from models.lag_ml_model import scaling_function,preprocess_data,classifier_models,train_model,grid_optimize_model,random_optimize_model,get_param_grids,get_data
from backtesting import Backtest,Strategy
import pandas as pd
def backtest(y_pred, X_test):
    def get_signal():
        print(type(y_pred))
        return y_pred
    class MyStrategy(Strategy):
        def init(self):
            self.signal = self.I(get_signal)
        def next(self):
            print(f"Signal: {self.signal[-1]}, Position: {self.position}")
            if self.signal[-1] == 1:
                if not self.position:
                    print("Executing Buy Order")
                    self.buy()
            elif self.signal[-1] == 0:
                if self.position:
                    print("Closing Position")
                    self.position.close()

    bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002,exclusive_orders=True)
    stats = bt.run()
    return stats,bt

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
    df = get_data(path=r'C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')
    X_train,X_test,y_train,y_test = preprocess_data(df,max_lag=60)
    X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
    normal_df,bt_collection,results,y_pred_dict = normal_model(X_train_scaled,X_test_scaled,y_train,y_test)
    print("Working")
    print(bt_collection)


In [2]:
df = get_data(path=r'C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')
X_train,X_test,y_train,y_test = preprocess_data(df)
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
classifier = classifier_models()
norm_results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
stats_l=[]
bt_collection = {}
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats,bt = backtest(y_pred,X_test)
    bt_collection[name] = bt
    stats['Algorithm'] = name
    stats_l.append(stats)

normal_df = pd.DataFrame(stats_l)
param_grids = get_param_grids()
print("Working on optimizing with GridSearchCV")
best_models = grid_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
grid_results,y_pred_dict = train_model(classifiers=best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
stats_l=[]
bt_collection = {}
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats,bt = backtest(y_pred,X_test)
    bt_collection[name] = bt
    stats['Algorithm'] = name
    stats_l.append(stats)

grid_df = pd.DataFrame(stats_l)
print("Working on optimizing with RandomSearchCV")
random_best_models = random_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
rand_results,y_pred_dict = train_model(classifiers=random_best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
stats_l=[]
bt_collection = {}
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats,bt = backtest(y_pred,X_test)
    bt_collection[name] = bt
    stats['Algorithm'] = name
    stats_l.append(stats)

random_df = pd.DataFrame(stats_l)



(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6328
Confusion Matrix:
[[279  76]
 [199 195]]
              precision    recall  f1-score   support

           0       0.58      0.79      0.67       355
           1       0.72      0.49      0.59       394

    accuracy                           0.63       749
   macro avg       0.65      0.64      0.63       749
weighted avg       0.66      0.63      0.63       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.6302
Confusion Matrix:
[[265  90]
 [187 207]]
              precision    recall  f1-score   support

           0       0.59      0.75      0.66       355
           1       0.70      0.53      0.60       394

    accuracy                           0.63       749
   macro avg       0.64      0.64      0.63       749
weighted avg       0.64      0.63      0.63       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



MLP Performance:
Accuracy: 0.8652
Confusion Matrix:
[[294  61]
 [ 40 354]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.85       355
           1       0.85      0.90      0.88       394

    accuracy                           0.87       749
   macro avg       0.87      0.86      0.86       749
weighted avg       0.87      0.87      0.86       749

[LightGBM] [Info] Number of positive: 1098, number of negative: 647
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002579 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16830
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.629226 -> initscore=0.528899
[LightGBM] [Info] Start training from score 0.528899
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

...

Backtesting for SGD Classifier
<class 'numpy.ndarray'>


Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 1, Position: <Position: 0 (0 trades)>
Executing Buy Order
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal

Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 1, Position: <Position: 0 (0 trades)>
Executing Buy Order
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal: 1, Position: <Position: 58 (1 trades)>
Signal

Backtesting for LGBMClassifier
<class 'numpy.ndarray'>


Signal: 1, Position: <Position: 0 (0 trades)>
Executing Buy Order
Signal: 1, Position: <Position: 62 (1 trades)>
Signal: 0, Position: <Position: 62 (1 trades)>
Closing Position
Signal: 0, Position: <Position: 0 (0 trades)>
Signal: 1, Position: <Position: 0 (0 trades)>
Executing Buy Order
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal: 1, Position: <Position: 66 (1 trades)>
Signal

In [7]:
rand_results

,Model,Accuracy,Confusion Matrix
0,Logistic Regression,0.889186,"[[312, 43], [40, 354]]"
1,Random Forest,0.612817,"[[298, 57], [233, 161]]"
2,XGBoost,0.539386,"[[10, 345], [0, 394]]"
3,SVM,0.886515,"[[319, 36], [49, 345]]"
4,Naive Bayes,0.526035,"[[0, 355], [0, 394]]"
5,Gradient Boosting,0.606142,"[[293, 62], [233, 161]]"
6,K-Nearest Neighbors,0.639519,"[[221, 134], [136, 258]]"
7,Decision Tree,0.634179,"[[131, 224], [50, 344]]"
8,AdaBoost,0.526035,"[[0, 355], [0, 394]]"
9,SGD Classifier,0.886515,"[[330, 25], [60, 334]]"


In [10]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [11]:
random_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Algorithm
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,17.061994,-1.019783,-0.086857,-0.117629,-0.061579,-21.921150,0.386970,-24.066100,-6.941420,344 days,89 days,24,25.000000,20.464318,-11.267103,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-04,2025-02-27,1091 days,30.440587,11916.425782,12240.163985,39.581954,19.164258,45.431149,6.076509,14.609957,4.132972,0.415916,0.664477,0.238469,7.156062,0.264316,-25.481290,-4.162446,608 days,96 days,10,40.000000,41.798618,-12.116535,1.801510,175 days,32 days,1.961056,2.616586,0.485667,0.168882,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,Random Forest
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,30.086406,8.248840,0.406773,0.672415,0.417197,-2.712739,0.960830,-29.334674,-6.804793,456 days,62 days,3,33.333333,0.189139,-18.102253,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,XGBoost
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,16.969569,4.259559,0.369154,0.577750,0.302364,3.914446,0.349501,-20.718028,-5.188582,316 days,75 days,21,33.333333,19.934187,-7.247327,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit...",Naive Bayes
5,2022-03-04,2025-02-27,1091 days,33.244326,12032.101503,12444.939077,105.656736,20.321015,45.431149,6.421844,14.149592,4.365591,0.453854,0.706241,0.251984,8.919305,0.250967,-25.485128,-3.915521,541 days,75 days,26,46.153846,23.050493,-5.340843,0.767003,85 days,13 days,1.763773,0.947239,0.734008,0.191108,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,20.075484,-1.999025,-0.144378,-0.204836,-0.113310,-33.877422,0.561431,-25.580016,-4.724991,577 days,77 days,57,45.614035,18.725328,-8.066670,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-04,2025-02-27,1091 days,74.499332,17483.827577,19069.386553,189.852669,74.838276,45.431149,20.679837,24.943631,13.774350,0.829063,1.540883,0.929257,49.088710,0.566782,-22.254163,-3.019263,497 days,28 days,34,58.823529,21.274586,-5.269782,1.931012,137 days,23 days,3.235849,2.084878,2.023089,0.391667,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Decision Tree
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,

In [12]:
grid_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Algorithm
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,17.061994,-1.019783,-0.086857,-0.117629,-0.061579,-21.921150,0.386970,-24.066100,-6.941420,344 days,89 days,24,25.000000,20.464318,-11.267103,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-04,2025-02-27,1091 days,44.993324,11114.328596,11125.638335,102.440714,11.143286,45.431149,3.618512,17.981624,2.470332,0.201234,0.308193,0.120519,-7.317890,0.406355,-30.024296,-11.027547,608 days,177 days,27,55.555556,47.320745,-14.071836,0.432415,188 days,17 days,1.436924,0.838544,0.296060,0.110796,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Random Forest
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,30.086406,8.248840,0.406773,0.672415,0.417197,-2.712739,0.960830,-29.334674,-6.804793,456 days,62 days,3,33.333333,0.189139,-18.102253,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,XGBoost
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,16.969569,4.259559,0.369154,0.577750,0.302364,3.914446,0.349501,-20.718028,-5.188582,316 days,75 days,21,33.333333,19.934187,-7.247327,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit...",Naive Bayes
5,2022-03-04,2025-02-27,1091 days,46.595461,13854.080345,14101.100279,146.901284,38.540803,45.431149,11.592143,16.743477,7.820595,0.692338,1.166237,0.529313,24.200241,0.315655,-21.900352,-3.745094,344 days,52 days,32,46.875000,43.337307,-4.790504,1.076605,186 days,15 days,2.095208,1.362134,0.880201,0.221592,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,20.075484,-1.999025,-0.144378,-0.204836,-0.113310,-33.877422,0.561431,-25.580016,-4.724991,577 days,77 days,57,45.614035,18.725328,-8.066670,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit...",Decision Tree
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exi

In [17]:
grid_df.rename(columns={'Algorithm':'Model'},inplace=True)

In [19]:
pd.merge(grid_results,grid_df, how='outer')

,Model,Accuracy,Confusion Matrix,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],CAGR [%],Sharpe Ratio,Sortino Ratio,Calmar Ratio,Alpha [%],Beta,Max. Drawdown [%],Avg. Drawdown [%],Max. Drawdown Duration,Avg. Drawdown Duration,# Trades,Win Rate [%],Best Trade [%],Worst Trade [%],Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,AdaBoost,0.526035,"[[0, 355], [0, 394]]",2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
1,Decision Tree,0.526035,"[[0, 355], [0, 394]]",2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
2,Gradient Boosting,0.619493,"[[251, 104], [181, 213]]",2022-03-04,2025-02-27,1091 days,46.595461,13854.080345,14101.100279,146.901284,38.540803,45.431149,11.592143,16.743477,7.820595,0.692338,1.166237,0.529313,24.200241,0.315655,-21.900352,-3.745094,344 days,52 days,32,46.875000,43.337307,-4.790504,1.076605,186 days,15 days,2.095208,1.362134,0.880201,0.221592,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
3,K-Nearest Neighbors,0.639519,"[[221, 134], [136, 258]]",2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,20.075484,-1.999025,-0.144378,-0.204836,-0.113310,-33.877422,0.561431,-25.580016,-4.724991,577 days,77 days,57,45.614035,18.725328,-8.066670,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,LGBMClassifier,0.636849,"[[87, 268], [4, 390]]",2022-03-04,2025-02-27,1091 days,61.682243,14097.946560,15379.425732,77.401360,40.979466,45.431149,12.249207,26.030064,8.256039,0.470579,0.778922,0.381868,7.473893,0.737502,-32.077034,-6.912803,664 days,75 days,20,45.000000,24.671346,-11.783238,-0.035170,288 days,32 days,1.097800,0.228987,-0.018232,-0.005564,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
5,Logistic Regression,0.889186,"[[312, 43], [40, 354]]",2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,17.061994,-1.019783,-0.086857,-0.117629,-0.061579,-21.921150,0.386970,-24.066100,-6.941420,344 days,89 days,24,25.000000,20.464318,-11.267103,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,MLP,0.829105,"[[267, 88], [40, 354]]",2022-03-04,2025-02-27,1091 days,59.679573,10828.594591,12014.472406,66.103784,8.285946,45.431149,2.714494,17.562831,1.855735,0.154559,0.228183,0.127745,-9.498867,0.391467,-21.249331,-6.279892,345 days,76 days,17,41.176471,18.779749,-8.376325,0.360918,147 days,37 days,1.260991,0.567991,0.239221,0.059862,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,Naive Bayes,0.526035,"[[0, 355], [0, 394]]",2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,30.916903,9.586894,0.461394,0.777863,0.479723,3.996758,0.982611,-29.735633,-5.702116,429 days,50 days,0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
8,Random Forest,0.623498,"[[256, 99], [183, 211]]",2022-03-04,2025-02-27,1091 days,44.993324,111